In [ ]:
from product_types import *
from precal_classes import *
from data_visualization import *
from copy import deepcopy

import pandas as pd 
from pathlib import Path
from importlib.machinery import SourceFileLoader

# Basic fossil fuels

In [ ]:
nat_gas = Fuel(name = "Natural Gas", carbon_frac = 0.763, processing = 0.258)
crude = Fuel(name = "Conventional Crude", carbon_frac = 0.85, processing = 0.126)
coal = Fuel("Bituminous Coal", 0.7, 0.117)

print(nat_gas)
print("GHG intensity (CO2e):", round(nat_gas.ggi_co2e(), 2))
print()
print(crude)
print("GHG intensity (CO2e):", round(crude.ggi_co2e(), 2))
print()
print(coal)
print("GHG intensity (CO2e):", round(coal.ggi_co2e(), 2))


# LNG

In [ ]:
lng = Product(name = "LNG", processing = 0)

lng.add_feedstock(nat_gas, unit_ratio = 1/.9)

print(lng)
print("GHG intensity (CO2e):", round(lng.ggi_co2e(), 2))


# Natural gas with leakage

Here, my result differs with the Excel spreadsheet. 
The formula for effective CO2 emissions of CH4 
leakage in the Excel spreadsheet depends on the 
molecular ratio between C and CO2. I think that is 
incorrect. Global warming attributable to CH4 
leakage does not involve the carbon fraction of the 
gas or the properties of its combustion, just its 
global warming potential.

I believe the Excel spreadsheet may have been
corrected since I wrote this, but I'll leave it in
for now just as a side note.

In [ ]:
leaky_nat_gas = deepcopy(nat_gas)

leaky_nat_gas.set_leakage(leak_rate = 0.01, gwp = 30)
leaky_nat_gas.name = "NG 1% leak"

print(leaky_nat_gas)
print("GHG intensity (CO2e):", round(leaky_nat_gas.ggi_co2e(), 2))


# A metal

Products take a processing value 

In [ ]:
steel = Product("BOF raw steel", 0)

# feedstocks
iron = Fuel("Iron ore", 0, .168)

coal = Fuel("Coal", 0.93/1.5, 0.034)

coke = Product("Coke", 0.496)
coke.add_feedstock(coal, unit_ratio = 1.5)

oxygen = Fuel("Oxygen", 0, .525)

limestone = Fuel("Limestone", 0, 0.44)

# update steel
steel.add_feedstock(iron, unit_ratio = 1.6)
steel.add_feedstock(coke, unit_ratio = 0.65)
steel.add_feedstock(oxygen, unit_ratio = 0.154)
steel.add_feedstock(limestone, unit_ratio = 0.25)

# calculate GGI
steel.ggi_co2e()


# 'Print catalog' functionality

In [ ]:
steel.catalog().show()


# Bar chart functionality

Currently only available for `Fuel` type

In [ ]:
plot = Plot(nat_gas, leaky_nat_gas, crude)
plot.column_plot()

In [ ]:
plot.add_column(coal)
plot.column_plot()

# Testing Class Edits 

In [ ]:
# setting named instances of the subclasses for feedstocks negates the catalog function output 

bof_steel = BOFSteel(iron, coke, limestone, oxygen, processing = 0)
tree = bof_steel.catalog()
tree.show()

tree2 = coke.catalog()
tree2.show()

## Data Input for Generating Classes 

In [ ]:
custom_BOF_steel_input = pd.DataFrame([
    {"name": "Iron Ore", "carbon_frac": 0.5, "processing": 2.0},
    {"name": "Coke", "carbon_frac": 0.4, "processing": 1.4}
])

# add column with type - fuel or product 
# need column mapping feedstocks to products 
# person making excel spreadsheet - product 1 
custom_BOF_Steel = pd.DataFrame([
    {
        "name": "BOF Steel",
        "processing": 0.5,
        # second item the carbon fraction 
        "feedstocks": [("Iron Ore", 1.5), ("Coke", 0.8)]
    }
])

print(custom_BOF_Steel)

fuel_dict = {row['name']: ModFuel.from_dataframe_row(row) for _, row in custom_BOF_steel_input.iterrows()}

EJ_BOF_Steel = {
    row['name']: ModProduct.from_dataframe_row(row, fuel_dict)
    for _, row in custom_BOF_Steel.iterrows()
}

steel_product = EJ_BOF_Steel["BOF Steel"]
print(steel_product.ggi_co2e())

## Data Input for Generating Classes - One Dataframe

In [ ]:
df = pd.DataFrame([
    {"product_name": "BOF Steel", "processing": 0.5, "feedstock_name": "Iron Ore", "feedstock_type": "Fuel", "carbon_frac": 0.5, "unit_ratio": 1.4},
    {"product_name": "BOF Steel", "processing": 0.5, "feedstock_name": "Coke", "feedstock_type": "Fuel", "carbon_frac": 0.4, "unit_ratio": 0.8},
    {"product_name": "Coke", "processing": 0.4, "feedstock_name": "Coal", "feedstock_type": "Fuel", "carbon_frac": 0.6, "unit_ratio": 0.4}
])

product_dict = build_products_from_df(df)

for name, prod in product_dict.items():
    print(f"{name}: GGI CO2e = {prod.ggi_co2e():.2f}")
    print(f"{name}: GGI CO2e = {prod.catalog()}")
    

### Taking Input from a CSV is also possible with this method 

In [ ]:
csv = Path("/Users/ejoiner/OneDrive - rff/Documents/RFF Organization/Research Documents/GGI/Test/test_data.csv")

df2 = pd.read_csv(csv)

product_dict2 = build_products_from_df(df2)

for name, prod in product_dict2.items():
    print(f"{name}: GGI CO2e = {prod.ggi_co2e():.2f}")
    print(f"{name}: GGI CO2e = {prod.catalog()}")
    

#### What's Next?

- Allowing products to have multiple sources of electricity and thermal energy, defined by percentages 
- Other initializations for core products 

- About 20 hours remaining of existing hours allotment, will work on implementing core features. 